# Optimizing an ACT-R Model

Here, we will see how to optimize an ACT-R model using Python libraries for scientific computing. We will use John Anderson's model of the Fan Effect from ACT-R's tutorial, Unit 5. First, let's load the necessary libraries

In [366]:
import actr
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

## The fan model

John Anderson's fan model is explained in detailed in the ACT-R tutorial, Unit 5. There are actually different versions of the model. The code used here is a modified version of the model called `fan-no-pm-model.lisp`, which does not use any Perceptual/Motor interface (and, therefore, requires no device). This is just to make it easy to interact with the code.

### Splitting the model

In this notebook, we will optimize the values of four parameters, `:LF`, `:MAS`, `:GA`, and `:IMAGINAL-ACTIVATION`. Being _decalarative_ parameters, they cannot be changed after chunks have been loaded into the model. So, we will actually split the model code into two parts. One file, `fan-no-pm-main.lisp`, contains the man model code, while the other file `fan-no-pm-chunks.lisp`, contains the declarative memory chunks. This permits us to execute the load the model in two steps, and modify the parameters in the middle.

### Model performance in the fan experiment

One of the quirks of this model is that there is no _noise_ whatsoever, so it does not require more than one run per condition. That means that precise estimates can be obtained by running only 18 trials of the model, one for each level of the fan effect (Fan of 1, 2, 3), from each source of activation (Person or Location), and for each item type (Target or Foil). For each possible combination of these factors, the function `sentence` sets the corresponding stimulus into the goal buffer and lets the model retrieve a response and record the answer. 

The function `sentence` takes 8 arguments, the first four of which specify the stimulus type (person fan, location fan, foil or target, and retrieval strategy) and the last four specificy the values of the four parameters that we want to change.

In [360]:
def sentence (person, location, target, term, 
              lf = 0.63, mas = 1.6, ga = 2.0, ia = 1.0):
    actr.load_act_r_model("fan-no-pm-main.lisp")

    actr.set_parameter_value(":lf", lf)
    actr.set_parameter_value(":mas", mas)
    actr.set_parameter_value(":ga", ga)
    actr.set_parameter_value(":imaginal-activation", ia)
    
    actr.load_act_r_code("fan-no-pm-chunks.lisp")
    
    if term == 'person':
        actr.pdisable("retrieve-from-location")
    else:
        actr.pdisable("retrieve-from-person")

    # Instead of presenting the items visibly 
    # just modify the chunk which will be placed 
    # into the goal buffer when the model runs.

    actr.mod_chunk("goal", "arg1", person, "arg2", location, "state", "test")

    response_time = actr.run(30)[0]
    response = actr.chunk_slot_value(actr.buffer_read("goal"),"state")


    # Return the list of the time and whether 
    # the correct answer was given.
      
    if target:
        if response.lower() == "'k'".lower():
            return (response_time ,True)
        else:
            return (response_time ,False)
    else:
        if response.lower() == "'d'".lower():
            return (response_time ,True)
        else:
            return (response_time ,False)
        

These functions provide a handy way to simulate an an entire experiment 

In [365]:
def do_person_location(term, lf, mas, ga, ia):
    data = []
    for person,location,target in [("'lawyer'", "'store'", True),
                                   ("'captain'", "'cave'", True),
                                   ("'hippie'", "'church'", True),
                                   ("'debutante'", "'bank'", True),
                                   ("'earl'", "'castle'", True),
                                   ("'hippie'", "'bank'", True),
                                   ("'fireman'", "'park'", True),
                                   ("'captain'", "'park'", True),
                                   ("'hippie'", "'park'", True),
                                   ("'fireman'", "'store'", False),
                                   ("'captain'", "'store'", False),
                                   ("'giant'", "'store'", False),
                                   ("'fireman'", "'bank'", False),
                                   ("'captain'", "'bank'", False),
                                   ("'giant'", "'bank'", False),
                                   ("'lawyer'", "'park'", False),
                                   ("'earl'", "'park'", False),
                                   ("'giant'", "'park'", False)]:

        data.append(sentence(person, location, target, term, 
                             lf, mas, ga, ia))

    return data

  
def experiment():
    output_person_location(list(map(lambda x,y:((x[0]+y[0])/2,(x[1] and y[1])),
                                    do_person_location('person'),
                                    do_person_location('location'))))
    
def output_person_location(data):
    """Prints the Person/Location data as a nice Fan Effect table"""
    rts = list(map(lambda x: x[0],data))
    actr.correlation(rts,person_location_data)
    actr.mean_deviation(rts,person_location_data)

    print("\nTARGETS:\n                         Person fan")
    print("  Location      1             2             3")
    print("    fan")
    
    for i in range(3):
        print("     %d      " % (i+1),end="")
        for j in range(3):
            print("%6.3f (%-5s)" % (data[j + (i * 3)]),end="")
        print()

    print()
    print("FOILS:")
    for i in range(3):
        print("     %d      " % (i+1),end="")
        for j in range(3):
            print("%6.3f (%-5s)" % (data[j + ((i + 3) * 3)]),end="")
        print()

# Optimization

Now, with the code in place to run the model, we can start thinking about optimizing it. First, we need to define our _ground truth_, that is, the empirical data we want to compare our model against. Here is the data from John Anderson's 1974 Fan Effect experiment. 

In [367]:
person_location_data = [1.11, 1.17, 1.22,
                        1.17, 1.20, 1.22,
                        1.15, 1.23, 1.36,
                        1.20, 1.22, 1.26,
                        1.25, 1.36, 1.29,
                        1.26, 1.47, 1.47]

## The objective function
We define the _objective function_ we want to _minimize_. In this case, the RMSE between predicted data (the model) and the observed data (the experiment).

In [362]:
def fan_rmse(lf, mas, ga, ia):
    """Calculates RMSE for Fan Effect data (objective fun to minimize)"""
    L = list(map(lambda x,y:((x[0]+y[0])/2,(x[1] and y[1])),
                        do_person_location('person', lf, mas, ga, ia),
                        do_person_location('location', lf, mas, ga, ia)))
    rts = [x[0] for x in L]
    # We do not care about accuracy data but, if we did, 
    # it would be analyzed here.
    # ACCs = [x[1] for x in L]
    R = np.array(rts)
    #print(rts)
    D = np.array(person_location_data)
    RMSE = np.sqrt(np.mean((D-R)**2))
    return(RMSE)

For example, here is the RMSE for the value used by John Anderson's classical study

In [368]:
fan_rmse(lf = 0.63, mas=1.6, ga=1.0, ia=1.0)

0.052779941476115956

And this is what happens when we pick some other values:

In [369]:
fan_rmse(lf = 2, mas=0, ga=0, ia=0)

1.591903996267782

# Minimization 
To minimize, we will use Scipy's optimize package and invoke the Nelder-Meade algorithm, which performs gradient descent without needing any information about the function derivatives (which is our case). The first step is to put our function in vector form, which is done by creating a _wrapper_ function that takes the ACT-R parameters as elements of a vector, instead of distinct arguments

In [355]:
def target_func(x):
    return(fan_rmse(x[0],  # :lf
                    x[1],  # :mas
                    x[2],  # :ga
                    x[3])) # :imaginal-activation

Then, we need to define a vector of initial values

In [356]:
init = np.array([1.0, 1.0, 1.0, 0.0])

Then, we need to define a set of reasonable bounds for each value. For instance, we do not want negative latency factors or spreading activations. Each bound is defined as a `(min, max)` tuple:

In [343]:
bounds = [(0, 5), (0, 5), (0, 5), (0, 5)]

In [370]:
target_func(init)

0.5236946735561773

In [373]:
minimize(target_func, init, method = "Nelder-Mead", options = {"maxiter" : 200})

 final_simplex: (array([[ 6.16606693e-01,  1.59431959e+00,  1.09249125e+00,
        -5.39659632e-04],
       [ 6.16613721e-01,  1.59434065e+00,  1.09246312e+00,
        -5.39496950e-04],
       [ 6.16580047e-01,  1.59428148e+00,  1.09244077e+00,
        -5.39268191e-04],
       [ 6.16593380e-01,  1.59431944e+00,  1.09247478e+00,
        -5.39556676e-04],
       [ 6.16607713e-01,  1.59433767e+00,  1.09246559e+00,
        -5.39551434e-04]]), array([0.04885864, 0.04885864, 0.04885864, 0.04885864, 0.04885864]))
           fun: 0.04885863963176489
       message: 'Optimization terminated successfully.'
          nfev: 155
           nit: 75
        status: 0
       success: True
             x: array([ 6.16606693e-01,  1.59431959e+00,  1.09249125e+00, -5.39659632e-04])